In [17]:
import os
import time
import ntpath
import numpy as np
from PIL import Image
from os.path import join, exists
from keras.models import model_from_json
from sklearn.metrics import mean_squared_error
import math
import random
from tqdm import tqdm

# Define the path to the dataset folders
datasets = [
    "D:/IITG/UW/DATASETS FOR COMPARISION/EUVP",
    "D:/IITG/UW/DATASETS FOR COMPARISION/UIEB",
    "D:/IITG/UW/DATASETS FOR COMPARISION/Degraded"
]

# Define the path to the output folder
output_folder = "C:/Users/raiha/Downloads/FUnIE-GAN/data/output"

# Function to enhance underwater images
def enhance_underwater_image(input_image, generator_model):
    im = preprocess(input_image)
    im = np.expand_dims(im, axis=0)
    
    # Generate enhanced image
    gen = generator_model.predict(im)
    gen_img = deprocess(gen)[0]
    
    return gen_img

# Function to process and enhance images in a dataset
def process_and_enhance_dataset(dataset_dir, output_dir, model, num_samples=890):
    test_paths = getPaths(dataset_dir)
    test_paths = list(test_paths)  # Convert to a list

    # Calculate the number of available images in the dataset
    num_available_images = len(test_paths)

    # Ensure that the number of samples does not exceed the available images
    num_samples = min(num_samples, num_available_images)

    # Randomly select a subset of images
    selected_test_paths = random.sample(test_paths, num_samples)

    # Create a directory for the current dataset in the output folder
    dataset_name = ntpath.basename(dataset_dir)
    dataset_output_dir = os.path.join(output_dir, dataset_name)
    if not exists(dataset_output_dir):
        os.makedirs(dataset_output_dir)

    # Initialize lists to store MSE and PSNR values for all images
    all_mse_values = []
    all_psnr_values = []

    # Testing loop with progress bar
    for img_path in tqdm(selected_test_paths, desc=f"Enhancing {dataset_name}"):
        inp_img = read_and_resize(img_path, (256, 256))

        # Enhance the image
        s = time.time()
        gen_img = enhance_underwater_image(inp_img, model)
        tot = time.time() - s
        times.append(tot)

        # Calculate MSE between the original image and enhanced image
        mse = mean_squared_error(inp_img.flatten(), gen_img.flatten())
        all_mse_values.append(mse)

        # Calculate PSNR using the calculated MSE
        psnr = 10 * math.log10((255 ** 2) / mse)
        all_psnr_values.append(psnr)

        # Save output images
        img_name = ntpath.basename(img_path)
        out_img = np.hstack((inp_img, gen_img)).astype('uint8')
        Image.fromarray(out_img).save(join(dataset_output_dir, img_name))

    # Print or store the MSE and PSNR values for each image
    for i, img_path in enumerate(selected_test_paths):
        img_name = ntpath.basename(img_path)
        print(f"Dataset: {dataset_name}, Image {i+1}: {img_name}, MSE: {all_mse_values[i]}, PSNR: {all_psnr_values[i]}")

# Load the model
checkpoint_dir = 'models/gen_p/'
model_name_by_epoch = "model_15320_"

model_h5 = checkpoint_dir + model_name_by_epoch + ".h5"
model_json = checkpoint_dir + model_name_by_epoch + ".json"

# Sanity check
assert (exists(model_h5) and exists(model_json))

with open(model_json, "r") as json_file:
    loaded_model_json = json_file.read()
funie_gan_generator = model_from_json(loaded_model_json)
funie_gan_generator.load_weights(model_h5)

# Iterate through the datasets and process/enhance random 890 images
for dataset_dir in datasets:
    process_and_enhance_dataset(dataset_dir, output_folder, funie_gan_generator)

# Some statistics
num_test = 890 * len(datasets)
if num_test == 0:
    print("\nFound no images for test")
else:
    print("\nTotal images: {0}".format(num_test))
    Ttime, Mtime = np.sum(times[1:]), np.mean(times[1:])
    print("Time taken: {0} sec at {1} fps".format(Ttime, 1. / Mtime))
    print("\nSaved generated images in {0}\n".format(output_folder))


Enhancing EUVP:   0%|                                                                          | 0/890 [00:00<?, ?it/s]

1/1 [==============================] - 0s 235ms/step


Enhancing EUVP:   0%|                                                                  | 1/890 [00:00<04:21,  3.40it/s]

1/1 [==============================] - 0s 77ms/step


Enhancing EUVP:   0%|▏                                                                 | 2/890 [00:00<02:55,  5.05it/s]

...

1/1 [==============================] - 0s 133ms/step


Enhancing Degraded: 100%|███████████████████████████████████████████████████████████▉| 889/890 [04:41<00:00,  3.05it/s]

1/1 [==============================] - 0s 127ms/step


Enhancing Degraded: 100%|████████████████████████████████████████████████████████████| 890/890 [04:42<00:00,  3.15it/s]

Dataset: Degraded, Image 1: frame_0222.png, MSE: 210.108154296875, PSNR: 24.906374531672625
Dataset: Degraded, Image 2: frame012_0441.png, MSE: 944.7046508789062, PSNR: 18.377843074296962
Dataset: Degraded, Image 3: frame019_0004.png, MSE: 519.6996459960938, PSNR: 20.973279398734043
Dataset: Degraded, Image 4: frame016_0624.png, MSE: 1090.2965087890625, PSNR: 17.755357394232583
Dataset: Degraded, Image 5: frame_0004.png, MSE: 433.8705139160156, PSNR: 21.757202246329065
Dataset: Degraded, Image 6: frame_0731.png, MSE: 66.35318756103516, PSNR: 29.91218569961105
Dataset: Degraded, Image 7: frame2_0283.png, MSE: 175.11692810058594, PSNR: 25.697522306307047
Dataset: Degraded, Image 8: frame_0696.png, MSE: 97.4657211303711, PSNR: 28.242284604671482
Dataset: Degraded, Image 9: frame_0566.png, MSE: 91.81034088134766, PSNR: 28.50188760987137
Dataset: Degraded, Image 10: frame_0093.png, MSE: 423.0312805175781, PSNR: 21.867078789446374
Dataset: Degraded, Image 11: frame2_0065.png, MSE: 99.5790786